<a href="https://colab.research.google.com/github/earl562/Hadza_Translation/blob/main/Hadza_Lang_Trans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Hadza Language Translation| GPU POOR 😞|Experimentation on Google Colab**
Before deploying the application, it's crucial to experiment and validate your models in a high-performance environment. Google Colab offers GPU resources that can expedite model training and evaluation.

pip install neccessary libraries.

**Tip 💸: It helps to use -q at the end of your pip install to reduce clutter.**

In [ ]:
!pip install transformers datasets torch evaluate pinecone-client arize chainlit -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.6 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification

# Load Swahili-English tokenizer and model
swahili_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
swahili_to_english_model = XLMRobertaForSequenceClassification.from_pretrained('swahili-english-model-path')

# Load Hadza-Swahili tokenizer and model
hadza_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
hadza_to_swahili_model = XLMRobertaForSequenceClassification.from_pretrained('hadza-swahili-model-path')



In [ ]:
from datasets import load_dataset
from transformers import DataCollatorWithPadding

# Load your Hadza-Swahili dataset
hadza_swahili_dataset = load_dataset('csv', data_files={'train': 'hadza_swahili_train.csv', 'validation': 'hadza_swahili_validation.csv'})

# Initialize the tokenizer
tokenizer = hadza_tokenizer

# Tokenize the dataset
def tokenize_hadza_swahili(examples):
    return tokenizer(examples['source'], truncation=True, padding='max_length', max_length=128)

tokenized_hadza_swahili = hadza_swahili_dataset.map(tokenize_hadza_swahili, batched=True)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results_hadza_swahili',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize Trainer
trainer_hadza_swahili = Trainer(
    model=hadza_to_swahili_model,
    args=training_args,
    train_dataset=tokenized_hadza_swahili['train'],
    eval_dataset=tokenized_hadza_swahili['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Start training
trainer_hadza_swahili.train()

hadza_to_swahili_model.save_pretrained('fine-tuned-hadza-swahili')
hadza_tokenizer.save_pretrained('fine-tuned-hadza-swahili')

